# Note

This is Fabien Crom's NB except that I increase the tSVD from 16 to 32 components. CV = 0.664 (from 0.662) and overall LB = 0.848 (fom 0.847).

The CITE predictions from this NB: https://www.kaggle.com/code/jsmithperera/msci-citeseq-quickstart-v3

# Multiome Quickstart With Sparse Matrices

This notebook is mostly for demonstrating the utility of sparse matrices in this competition. (Especially for the Multiome dataset).

As the Multiome dataset is  very sparse (about 98% of cells are zeros), it benefits greatly from being encoded as sparse matrices. 

This notebook is largely based on [this notebook](https://www.kaggle.com/code/ambrosm/msci-multiome-quickstart) by AmbrosM. It is a nice first attempt at handling Multiome data, and I thought it would informative for kagglers to be able to contrast directly the performances of sparse vs dense representations. 

Mostly, the differences with AmbrosM's notebooks are:
- We use a representation of the data in sparse CSR format, which let us load all of the training data in memory (using less than 8GB memory instead of the >90GB it would take to represent the data in a dense format)
- We perform PCA (actually, TruncatedSVD) on the totality of the training data (while AmbrosM's notebook had to work with a subset of 6000 rows and 4000 columns). 
- We keep 16 components (vs 4 in AmbrosM's notebook)
- We apply Ridge regression on 50000 rows (vs 6000 in AmbrosM's notebook)
- Despite using much more data, this notebook should run in a bit more than 10 minutes (vs >1h for AmbrosM's notebook)

The competition data is pre-encoded as sparse matrices in [this dataset](https://www.kaggle.com/datasets/fabiencrom/multimodal-single-cell-as-sparse-matrix) generated by [this notebook](https://www.kaggle.com/code/fabiencrom/multimodal-single-cell-creating-sparse-data/).

Since we will only generate the multiome predictions in this notebook, I am taking the CITEseq predictions from [this notebook](https://www.kaggle.com/code/vuonglam/lgbm-baseline-optuna-drop-constant-cite-task) by VuongLam, which is the public notebook with the best score at the time I am publishing.

In [1]:
import os, gc, pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from colorama import Fore, Back, Style
from matplotlib.ticker import MaxNLocator

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, scale
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn.metrics import mean_squared_error

import scipy
import scipy.sparse

In [3]:
# ! pip install sklearn, colorama

# The scoring function (from AmbrosM)

This competition has a special metric: For every row, it computes the Pearson correlation between y_true and y_pred, and then all these correlation coefficients are averaged.

In [4]:
def correlation_score(y_true, y_pred):
    """Scores the predictions according to the competition rules. 
    
    It is assumed that the predictions are not constant.
    
    Returns the average of each sample's Pearson correlation coefficient"""
    if type(y_true) == pd.DataFrame: y_true = y_true.values
    if type(y_pred) == pd.DataFrame: y_pred = y_pred.values
    if y_true.shape != y_pred.shape: raise ValueError("Shapes are different.")
    corrsum = 0
    for i in range(len(y_true)):
        corrsum += np.corrcoef(y_true[i], y_pred[i])[1, 0]
    return corrsum / len(y_true)


# Preprocessing and cross-validation

We first load all of the training input data for Multiome. It should take less than a minute.

In [6]:
%%time
train_inputs = scipy.sparse.load_npz("/home/mks/PycharmProjects/multimodal_single_cell_integration/dataset/sparce/train_multi_inputs_values.sparse.npz")

CPU times: user 20.2 s, sys: 578 ms, total: 20.8 s
Wall time: 20.8 s


## PCA / TruncatedSVD
It is not possible to directly apply PCA to a sparse matrix, because PCA has to first "center" the data, which destroys the sparsity. This is why we apply `TruncatedSVD` instead (which is pretty much "PCA without centering"). It might be better to normalize the data a bit more here, but we will keep it simple.

In [8]:
%%time
pca = TruncatedSVD(n_components = 32, random_state = 1)
train_inputs = pca.fit_transform(train_inputs)

CPU times: user 4min 13s, sys: 5.87 s, total: 4min 19s
Wall time: 4min 9s


## Random row selection and conversion of the target data to a dense matrix

Unfortunately, although sklearn's `Ridge` regressor do accept sparse matrices as input, it does not accept sparse matrices as target values. This means we will have to convert the targets to a dense format. Although we could fit in memory both the dense target data and the sparse input data, the Ridge regression process would then lack memory. Therefore, from now on, we will work with a subset of 50 000 rows from the training data.

In [9]:
np.random.seed(42)
all_row_indices = np.arange(train_inputs.shape[0])
np.random.shuffle(all_row_indices)
selected_rows_indices = all_row_indices[:50000]

In [10]:
train_inputs = train_inputs[selected_rows_indices]

In [12]:
%%time
train_target = scipy.sparse.load_npz("/home/mks/PycharmProjects/multimodal_single_cell_integration/dataset/sparce/train_multi_targets_values.sparse.npz")

CPU times: user 9.41 s, sys: 256 ms, total: 9.67 s
Wall time: 9.67 s


In [13]:
train_target = train_target[selected_rows_indices]
train_target = train_target.todense()
gc.collect()

1041

## KFold Ridge regression
`sklearn` complains that we should use array instead of matrices. Unfortunately, the old `scipy` version available on kaggle do not provide sparse arrays; only sparse matrices. So we suppress the warnings.

In [14]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

This Kfold ridge regression code is mostly taken from AmbrosM's [notebook](https://www.kaggle.com/code/ambrosm/msci-multiome-quickstart). Note that `sklearn`'s `Ridge` handles sparse matrices transparently. I found [this blog post](https://dziganto.github.io/Sparse-Matrices-For-Efficient-Machine-Learning/) that list the other algorithms of `sklearn` that accept sparse matrices.

In [15]:
%%time
# Cross-validation

kf = KFold(n_splits=3, shuffle=True, random_state=1)
score_list = []
for fold, (idx_tr, idx_va) in enumerate(kf.split(train_inputs)):
    model = None
    gc.collect()
    X_tr = train_inputs[idx_tr] # creates a copy, https://numpy.org/doc/stable/user/basics.copies.html
    y_tr = train_target[idx_tr]
    del idx_tr

    model = Ridge(copy_X=False)
    model.fit(X_tr, y_tr)
    del X_tr, y_tr
    gc.collect()

    # We validate the model
    X_va = train_inputs[idx_va]
    y_va = train_target[idx_va]
    del idx_va
    y_va_pred = model.predict(X_va)
    mse = mean_squared_error(y_va, y_va_pred)
    corrscore = correlation_score(y_va, y_va_pred)
    del X_va, y_va

    print(f"Fold {fold}: mse = {mse:.5f}, corr =  {corrscore:.3f}")
    score_list.append((mse, corrscore))

# Show overall score
result_df = pd.DataFrame(score_list, columns=['mse', 'corrscore'])
print(f"{Fore.GREEN}{Style.BRIGHT}{train_inputs.shape} Average  mse = {result_df.mse.mean():.5f}; corr = {result_df.corrscore.mean():.3f}{Style.RESET_ALL}")


Fold 0: mse = 2.02330, corr =  0.664
Fold 1: mse = 2.02389, corr =  0.663
Fold 2: mse = 2.02415, corr =  0.664
(50000, 32) Average  mse = 2.02378; corr = 0.664
CPU times: user 28.3 s, sys: 18.6 s, total: 46.9 s
Wall time: 19.9 s


# Retraining


In [16]:
# We retrain the model and then delete the training data, which is no longer needed
model, score_list, result_df = None, None, None # free the RAM occupied by the old model
gc.collect()
model = Ridge(copy_X=False) # we overwrite the training data
model.fit(train_inputs, train_target)


Ridge(copy_X=False)

In [17]:
del train_inputs, train_target # free the RAM
_ = gc.collect()

# Predicting

In [18]:
%%time
multi_test_x = scipy.sparse.load_npz("/home/mks/PycharmProjects/multimodal_single_cell_integration/dataset/sparce/test_multi_inputs_values.sparse.npz")
multi_test_x = pca.transform(multi_test_x)
test_pred = model.predict(multi_test_x)
del multi_test_x
gc.collect()

CPU times: user 24.5 s, sys: 16.7 s, total: 41.2 s
Wall time: 23.7 s


55

# Creating submission

We load the cells that will have to appear in submission.

In [22]:
! pip install fastparquet

  Using cached fastparquet-0.8.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)


In [23]:
%%time
# Read the table of rows and columns required for submission
eval_ids = pd.read_parquet("/home/mks/PycharmProjects/multimodal_single_cell_integration/dataset/sparce/evaluation.parquet")

# Convert the string columns to more efficient categorical types
#eval_ids.cell_id = eval_ids.cell_id.apply(lambda s: int(s, base=16))

eval_ids.cell_id = eval_ids.cell_id.astype(pd.CategoricalDtype())
eval_ids.gene_id = eval_ids.gene_id.astype(pd.CategoricalDtype())

CPU times: user 8.4 s, sys: 1.4 s, total: 9.8 s
Wall time: 9.82 s


In [24]:
# Prepare an empty series which will be filled with predictions
submission = pd.Series(name='target',
                       index=pd.MultiIndex.from_frame(eval_ids), 
                       dtype=np.float32)
submission

row_id    cell_id       gene_id        
0         c2150f55becb  CD86              NaN
1         c2150f55becb  CD274             NaN
2         c2150f55becb  CD270             NaN
3         c2150f55becb  CD155             NaN
4         c2150f55becb  CD112             NaN
                                           ..
65744175  2c53aa67933d  ENSG00000134419   NaN
65744176  2c53aa67933d  ENSG00000186862   NaN
65744177  2c53aa67933d  ENSG00000170959   NaN
65744178  2c53aa67933d  ENSG00000107874   NaN
65744179  2c53aa67933d  ENSG00000166012   NaN
Name: target, Length: 65744180, dtype: float32

We load the `index`  and `columns` of the original dataframe, as we need them to make the submission.

In [26]:
%%time
y_columns = np.load("/home/mks/PycharmProjects/multimodal_single_cell_integration/dataset/sparce/train_multi_targets_idxcol.npz",
                   allow_pickle=True)["columns"]

test_index = np.load("/home/mks/PycharmProjects/multimodal_single_cell_integration/dataset/sparce/test_multi_inputs_idxcol.npz",
                    allow_pickle=True)["index"]

CPU times: user 27.5 ms, sys: 7.94 ms, total: 35.5 ms
Wall time: 32.5 ms


We assign the predicted values to the correct row in the submission file.

In [27]:
cell_dict = dict((k,v) for v,k in enumerate(test_index)) 
assert len(cell_dict)  == len(test_index)

gene_dict = dict((k,v) for v,k in enumerate(y_columns))
assert len(gene_dict) == len(y_columns)

In [28]:
eval_ids_cell_num = eval_ids.cell_id.apply(lambda x:cell_dict.get(x, -1))
eval_ids_gene_num = eval_ids.gene_id.apply(lambda x:gene_dict.get(x, -1))

valid_multi_rows = (eval_ids_gene_num !=-1) & (eval_ids_cell_num!=-1)

In [29]:
submission.iloc[valid_multi_rows] = test_pred[eval_ids_cell_num[valid_multi_rows].to_numpy(),
eval_ids_gene_num[valid_multi_rows].to_numpy()]

In [30]:
del eval_ids_cell_num, eval_ids_gene_num, valid_multi_rows, eval_ids, test_index, y_columns
gc.collect()

939

In [31]:
submission

row_id    cell_id       gene_id        
0         c2150f55becb  CD86                    NaN
1         c2150f55becb  CD274                   NaN
2         c2150f55becb  CD270                   NaN
3         c2150f55becb  CD155                   NaN
4         c2150f55becb  CD112                   NaN
                                             ...   
65744175  2c53aa67933d  ENSG00000134419    6.206675
65744176  2c53aa67933d  ENSG00000186862    0.045758
65744177  2c53aa67933d  ENSG00000170959    0.032102
65744178  2c53aa67933d  ENSG00000107874    1.350811
65744179  2c53aa67933d  ENSG00000166012    5.112196
Name: target, Length: 65744180, dtype: float32

# Merging with CITEseq predictions

We use the CITEseq predictions from [this notebook](https://www.kaggle.com/code/vuonglam/lgbm-baseline-optuna-drop-constant-cite-task) by VuongLam.

In [33]:
submission.reset_index(drop = True, inplace = True)
submission.index.name = 'row_id'

# with open("partial_submission_multi.pickle", 'wb') as f:
#     pickle.dump(submission, f)
# submission

In [34]:
submission

row_id
0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
              ...   
65744175    6.206675
65744176    0.045758
65744177    0.032102
65744178    1.350811
65744179    5.112196
Name: target, Length: 65744180, dtype: float32

In [35]:
submission.to_csv('/home/mks/PycharmProjects/multimodal_single_cell_integration/submissions/multiome/w-sparce-m-tsvd-ridge_notebook.csv')

# Add cite predictions

In [23]:
#cite_submission = pd.read_csv("../input/lgbm-baseline-optuna-drop-constant-cite-task/submission.csv")

with open("../input/msci-citeseg-by-cell-type/citeseq_pred_by_cell.pickle", 'rb') as f: cite_pred = pickle.load(f)
    
#cite_submission = cite_submission.set_index("row_id")
#cite_submission = cite_submission["target"]

In [24]:
#submission[submission.isnull()] = cite_submission[submission.isnull()]
submission.iloc[:len(cite_pred.ravel())] = cite_pred.ravel()

In [25]:
submission

row_id
0           0.428702
1           0.577249
2           0.762736
3           4.797713
4           4.675150
              ...   
65744175    6.206671
65744176    0.045758
65744177    0.032102
65744178    1.350810
65744179    5.112195
Name: target, Length: 65744180, dtype: float32

In [26]:
submission.isnull().any()

False

In [27]:
submission.to_csv("submission.csv")

In [28]:
!head submission.csv

row_id,target
0,0.42870185
1,0.5772492
2,0.76273614
3,4.7977133
4,4.6751504
5,6.4788637
6,1.0631003
7,0.4752991
8,0.7362693
